In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm

In [2]:
%pip install torchsummary
from torchsummary import summary

Note: you may need to restart the kernel to use updated packages.


In [3]:
train_transforms = transformsCompose([
                                        transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                                        transforms.RandomApply(torch.nn.ModuleList([

                                          transforms.GaussianBlur(3, sigma=(0.1, 2.0)),
                                          transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),

                                        ]), p=0.2),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.1307,), (0.3081,))
                                      ])

test_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                    ])

In [4]:
train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

In [5]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

CUDA Available? True


In [61]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv0 = nn.Sequential(

            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3, 3), padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(8),

        )

        self.conv1 = nn.Sequential(

            nn.Conv2d(in_channels=8, out_channels=8, kernel_size=(3, 3), padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(8),

        ) 

        self.max_pool0 = nn.MaxPool2d(kernel_size=(2, 2))

        self.conv2 = nn.Sequential(

            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.2)

        )

        self.conv3 = nn.Sequential(

            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.2)

        ) 

        self.transition0 = nn.Conv2d(in_channels=16, out_channels=8, kernel_size=(1, 1), padding=0)
        self.max_pool1 = nn.MaxPool2d(kernel_size=(2, 2))

        self.conv4 = nn.Sequential(

            nn.Conv2d(in_channels=8, out_channels=32, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.2)

        )

        self.gap = nn.AvgPool2d(kernel_size=2)

        self.conv6 = nn.Sequential(

            nn.Conv2d(in_channels=32, out_channels=10, kernel_size=(2, 2), padding=0)

        )

    def forward(self, x):

        x = self.conv0(x)
        x = self.conv1(x)

        x = self.max_pool0(x)

        x = self.conv2(x)
        x = self.conv3(x)

        x = self.max_pool1(x)
        x = self.transition0(x)

        x = self.conv4(x)

        x = self.gap(x)

        x = self.conv6(x)

        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [62]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
            Conv2d-4            [-1, 8, 24, 24]             584
              ReLU-5            [-1, 8, 24, 24]               0
       BatchNorm2d-6            [-1, 8, 24, 24]              16
         MaxPool2d-7            [-1, 8, 12, 12]               0
            Conv2d-8           [-1, 16, 10, 10]           1,168
              ReLU-9           [-1, 16, 10, 10]               0
      BatchNorm2d-10           [-1, 16, 10, 10]              32
          Dropout-11           [-1, 16, 10, 10]               0
           Conv2d-12             [-1, 16, 8, 8]           2,320
             ReLU-13             [-1, 16, 8, 8]               0
      BatchNorm2d-14             [

In [63]:
train_acc = []
train_loss = []
test_acc = []
test_losses = []

def train(model, device, train_loader, optimizer, epoch):

  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0

  for batch, (data, target) in enumerate(pbar):

    data, target = data.to(device), target.to(device)

    optimizer.zero_grad()

    y_pred = model(data)

    loss = F.nll_loss(y_pred, target)

    loss.backward()
    optimizer.step()

    pred = y_pred.argmax(dim=1, keepdim=True)
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):

  model.eval()
  test_loss = 0
  correct = 0

  with torch.no_grad():
    for data, target in test_loader:

      data, target = data.to(device), target.to(device)
      y_pred = model(data)

      loss = F.nll_loss(y_pred, target, reduction='sum').item()
      pred = y_pred.argmax(dim=1, keepdim=True)

      correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    test_acc.append(100. * correct / len(test_loader.dataset))

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

In [64]:
model =  Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

EPOCHS = 15
for epoch in range(EPOCHS):

    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.12398997694253922 Batch_id=468 Accuracy=89.47: 100%|██████████| 469/469 [00:10<00:00, 44.10it/s] 



Test set: Average loss: 0.0000, Accuracy: 9795/10000 (97.95%)

EPOCH: 1


Loss=0.05607730522751808 Batch_id=468 Accuracy=97.17: 100%|██████████| 469/469 [00:10<00:00, 42.88it/s] 



Test set: Average loss: 0.0000, Accuracy: 9866/10000 (98.66%)

EPOCH: 2


...


Test set: Average loss: 0.0000, Accuracy: 9935/10000 (99.35%)

EPOCH: 13


Loss=0.049156319350004196 Batch_id=468 Accuracy=98.73: 100%|██████████| 469/469 [00:10<00:00, 43.18it/s] 



Test set: Average loss: 0.0000, Accuracy: 9940/10000 (99.40%)

EPOCH: 14


Loss=0.025669308379292488 Batch_id=468 Accuracy=98.66: 100%|██████████| 469/469 [00:11<00:00, 42.56it/s] 



Test set: Average loss: 0.0000, Accuracy: 9937/10000 (99.37%)

